**CITIZEN ENGAGEMENT PLATFORM**

**an other model: AI assistant regarding land related questions**

In [ ]:
!pip install pdfminer.six
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompati

In [ ]:
 # Example using pdfminer.six to extract text from a PDF
import pdfminer.high_level
import os
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
#Create a new folder named 'my_new_data'
os.makedirs("/kaggle/working/my_new_data", exist_ok=True)

In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extracts text content from a PDF file."""
    try:
        text = pdfminer.high_level.extract_text(pdf_path)
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return None

# Path to the uploaded legal document folder in Kaggle
# Replace 'legal-documents' with the actual uploaded folder name
document_folder = "/kaggle/input/legal-docs/legal_documents"

# Dictionary to store extracted text
all_documents_text = {}

# Check if the folder exists
if os.path.exists(document_folder):
    for filename in os.listdir(document_folder):
        if filename.lower().endswith('.pdf'):
            filepath = os.path.join(document_folder, filename)
            print(f"Processing: {filename}")
            text_content = extract_text_from_pdf(filepath)
            if text_content:
                all_documents_text[filename] = text_content
else:
    print(f"Document folder not found: {document_folder}")

Processing: Revised_National_Land_Policy-Final_Version_2019.pdf
Processing: Law Governing Land in Rwanda 2013.pdf
Processing: Water law gazetted,2018.pdf
Processing: 2024_Ministerial_Order_relating_to_water_use_permit_2024.pdf
Processing: 6. National_land_policy_english_version_ 2004.pdf
Processing: Rwanda Environmental Policy_English 2003.pdf
Processing: Law governing land 2021.pdf
Processing: Law_on_environment.pdf
Processing: National_Land-Use_and_Development_Master_Plan_2020-2050.pdf


In [ ]:
def clean_legal_text(text):
        """Cleans legal document text."""
        # Remove headers/footers patterns (example, will need adjustment based on document patterns)
        text = re.sub(r'Page \d+ of \d+', '', text)
        text = re.sub(r'\n\s*\n', '\n', text) # Normalize multiple newlines
        text = re.sub(r'\s+', ' ', text) # Replace multiple spaces with single space
        text = text.strip()
        return text

In [ ]:
cleaned_documents = {}
for filename, text in all_documents_text.items():
    cleaned_documents[filename] = clean_legal_text(text)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize a text splitter. It tries to split by paragraphs, then sentences, then words.
    # `chunk_size` and `chunk_overlap` are important parameters to tune.
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, # Number of characters per chunk
        chunk_overlap=150 # Number of characters to overlap between chunks
    )

chunks = []
    # Iterate through each cleaned document and split it into chunks
for filename, text in cleaned_documents.items():
    # Add metadata (like filename) to each chunk, useful for citing sources
    document_chunks = text_splitter.create_documents([text], metadatas=[{"source": filename}])
    chunks.extend(document_chunks)

print(f"\nTotal number of text chunks created: {len(chunks)}")


Total number of text chunks created: 1701


In [ ]:
    # Example using OpenAI's embeddings (requires an API key)

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Use a free HuggingFace model for embeddings
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create vector store
vectorstore = Chroma.from_documents(chunks, embeddings_model)


/tmp/ipykernel_36/1660128586.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
2025-08-22 13:31:02.321527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755869462.552125      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755869462.621768      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attemptin

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Example using a Free/Open Source Embedding Model & Local LLM
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import pipeline

# 1. Initialize HuggingFace Embeddings (free, no API key required)
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Assuming you already have `chunks` (list of langchain Document objects)
vectorstore = Chroma.from_documents(chunks, embeddings_model)

# 2. Initialize a free local LLM pipeline (e.g., 'google/flan-t5-large' or 'tiiuae/falcon-7b-instruct')
generator = pipeline("text2text-generation", model="google/flan-t5-large", device=-1)  # -1 = CPU, 0 = GPU
llm = HuggingFacePipeline(pipeline=generator)

# 3. Define a prompt template
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer based on the context, just say that you don't know, don't try to make up an answer.
Provide a precise and surgically explained answer, referencing the source document for each piece of information. Give your answer in
three different languages as found in legal documents; of the language is not available please don't make up the answer.

Context:
{context}

Question: {question}

Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_template)

# 4. Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True
)

# 5. Run a sample query
query = "when expropriation is done?"
print("\nGenerating answer...")
result = qa_chain.invoke({"query": query})

print("\n--- AI Assistant's Answer ---")
print("Answer:", result['result'])

# Show sources if available
if 'source_documents' in result:
    print("\n--- Sources Consulted ---")
    for doc in result['source_documents']:
        print(f"- {doc.metadata.get('source')}: (Page content snippet starting with '{doc.page_content[:100]}...')")
    print("-------------------------")


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipykernel_36/2810544293.py:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator)
Token indices sequence length is longer than the specified maximum sequence length for this model (990 > 512). Running this sequence through the model will result in indexing errors



Generating answer...

--- AI Assistant's Answer ---
Answer: 16o expropriation: an action based on the powers of the State, public institutions decentralized and entities with legal personality to remove a person from his or her property due to public interest upon prior fair compensation; payment of 18 et des dotées 16o expropriation: action basée sur le pouvoir de l’État, des institutions entités publiques décentralisées la de personnalité juridique qui consiste à retirer une personne de sa propriété à des fins d’intérêt général moyennant le paiement préalable juste indemnisation; d’une Official

--- Sources Consulted ---
- 6. National_land_policy_english_version_ 2004.pdf: (Page content snippet starting with 'trends. These zones have no infrastructure and urban services/facilities; b. Policy statements - Urb...')
- 6. National_land_policy_english_version_ 2004.pdf: (Page content snippet starting with 'trends. These zones have no infrastructure and urban services/facilities; b. Polic